In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from tqdm import tqdm_notebook
import numpy as np
from collections import deque

In [ ]:
#discount factor for future utilities
DISCOUNT_FACTOR = 0.99
gamma = 0.99
#number of episodes to run
NUM_EPISODES = 10000

#max steps per episode
MAX_STEPS = 10000

#score agent needs for environment to be solved
SOLVED_SCORE = 475

#device to run model on
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class PolicyNetwork(nn.Module):
  def __init__(self, observation_space, action_space,seed,layer_size):
        super(PolicyNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.input_layer = nn.Linear(observation_space, layer_size)
        self.output_layer = nn.Linear(layer_size, action_space)
  def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        actions = self.output_layer(x)
        action_probs = F.softmax(actions, dim=-1)
        return action_probs

In [ ]:

class StateValueNetwork(nn.Module):
    def __init__(self, observation_space,seed,layer_size):
        super(StateValueNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.input_layer = nn.Linear(observation_space, layer_size)
        self.output_layer = nn.Linear(layer_size, 1)
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        state_value = self.output_layer(x)
        return state_value

In [ ]:
def process_rewards(rewards,gamma):
    G = []
    total_r = 0
    for r in reversed(rewards):
        total_r = r + total_r * DISCOUNT_FACTOR
        G.insert(0, total_r)
    G = torch.tensor(G)
    #G = (G - G.mean())/G.std()
    return G

In [ ]:
def train_value(G, state_vals, optimizer):
    val_loss = F.mse_loss(state_vals, G)
    optimizer.zero_grad()
    val_loss.backward()
    optimizer.step()

In [ ]:
#Make environment
#env = gym.make('Acrobot-v1')
env = gym.make('CartPole-v1')
#Init network
'''
policy_network = PolicyNetwork(env.observation_space.shape[0], env.action_space.n)
stateval_network = StateValueNetwork(env.observation_space.shape[0])'''


#Init optimizer


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


'\npolicy_network = PolicyNetwork(env.observation_space.shape[0], env.action_space.n)\nstateval_network = StateValueNetwork(env.observation_space.shape[0])'

In [ ]:
pip install scikit-optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.0 MB/s eta 0:00:00


In [ ]:
seed_list = [1,42,30,25,17]

In [ ]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

In [ ]:
from skopt import gp_minimize

# Define the objective function
def objective(params):

    regret_avg = 0
    print(params,"Its just getting started ")
    for i in range(5):
      seed = seed_list[i]
      policy_network = PolicyNetwork(env.observation_space.shape[0], env.action_space.n,seed,params["network_size"])
      #stateval_network = StateValueNetwork(env.observation_space.shape[0],seed,params["network_size"])
      reset_weights(policy_network)
      #reset_weights(stateval_network)
      policy_optimizer = optim.Adam(policy_network.parameters(), params["LR"])
      #stateval_optimizer = optim.Adam(stateval_network.parameters(), params["LR"])
      ep = 0
      action_space = np.arange(env.action_space.n)
      total_rewards = []
      regret = 0
      while ep < NUM_EPISODES:
        state = env.reset()
        states = []
        rewards = []
        actions = []
        done = False
        while done == False:
          state = torch.from_numpy(state).float().unsqueeze(0)[0]
          action_probability = policy_network.forward(state).detach().numpy()
          if np.isnan(action_probability).any():
            return -float('inf')
          else:
            action = np.random.choice(action_space,p=action_probability)
            state.detach()
            next_state,r,done,_ = env.step(action)
            states.append(state)
            rewards.append(r)
            actions.append(action)
            if done :
              break
            state = next_state

        total_rewards.append(sum(rewards))
        G = process_rewards(rewards,gamma)
        G = torch.FloatTensor(G)

        rewards = torch.FloatTensor(rewards)

        policy_optimizer.zero_grad()
        deltas = [gt for gt in zip(G)]
        deltas = torch.tensor(deltas)
        logprob = [torch.log(policy_network.forward(states[i])) for i in range(len(deltas))]
        policy_loss = []
        for i in range(len(deltas)):

              d = deltas[i]

              lp = logprob[i][actions[i]]

              policy_loss.append(-d * lp)
        policy_optimizer.zero_grad()
        #print(policy_loss,len(actions))
        sum(policy_loss).backward()
        policy_optimizer.step()
        avg_rewards = np.mean(total_rewards[-100:])
        ep +=1
        if ep % 400  == 0:
          print("Ep:",ep,"last 100 episodes reward is  :",avg_rewards, end="\n")
        if avg_rewards > 475:
            print("problem solved at episode",ep)
            break
        regret += 475 - avg_rewards
      print(params,regret)
      regret_avg += regret
    print("regret_avg:",regret_avg,"for",params)
    return regret_avg








In [ ]:
'''
from skopt import gp_minimize
from skopt.space import Real, Categorical

# Define the search space
space = [Real(0.0001, 0.01, name='lr'),  # Continuous parameter from 0.01 to 0.0001
         Categorical(categories=[16, 32, 64, 128, 256], name='layer_size')]   #
         '''

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\nfrom skopt import gp_minimize\nfrom skopt.space import Real, Categorical\n\n# Define the search space\nspace = [Real(0.0001, 0.01, name='lr'),  # Continuous parameter from 0.01 to 0.0001\n         Categorical(categories=[16, 32, 64, 128, 256], name='layer_size')]   #\n         "

In [ ]:
#bounds = [(1e-4, 1e-2),]
'''
# Run the Bayesian optimization
res = gp_minimize(objective, space, n_calls=10, random_state=0)
print(res)
# Print the best parameters
print(f"Best parameters: learning rate = {res.x[0]}")
'''

'\n# Run the Bayesian optimization\nres = gp_minimize(objective, space, n_calls=10, random_state=0)\nprint(res)\n# Print the best parameters\nprint(f"Best parameters: learning rate = {res.x[0]}")\n'

In [ ]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

def main():
    wandb.init(project="RLA2cartepole-wobase")
    #agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0, config=wandb.config)
    score = objective(wandb.config)
    wandb.log({"score": score})

# 2: Define the search space
sweep_configuration = {
    "method": "bayes",

    "metric": {"goal": "minimize", "name": "score"},
    "parameters": {
        "LR": {"max": 1e-2, "min": 1e-5},
        "network_size": {"values": [64,256,128]},
    },
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="RLA2cartepole-wobase")

wandb.agent(sweep_id, function=main, count=10)

/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Create sweep with ID: hmh76vmf
Sweep URL: https://wandb.ai/deep_learning_duri/RLA2cartepole-wobase/sweeps/hmh76vmf


wandb: Agent Starting Run: gpxzb2l9 with config:
wandb: 	LR: 0.004645806369298588
wandb: 	network_size: 128
wandb: Currently logged in as: dhurilkun (deep_learning_duri). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'LR': 0.004645806369298588, 'network_size': 128} Its just getting started 


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


...

wandb: Agent Starting Run: j6iyjx50 with config:
wandb: 	LR: 0.006853077550144784
wandb: 	network_size: 256
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'LR': 0.006853077550144784, 'network_size': 256} Its just getting started 
Ep: 400 last 100 episodes reward is  : 87.39
Ep: 800 last 100 episodes reward is  : 9.35
Ep: 1200 last 100 episodes reward is  : 9.44
Ep: 1600 last 100 episodes reward is  : 9.39
Ep: 2000 last 100 episodes reward is  : 9.33
Ep: 2400 last 100 episodes reward is  : 9.38
Ep: 2800 last 100 episodes reward is  : 9.35
Ep: 3200 last 100 episodes reward is  : 9.59
Ep: 3600 last 100 episodes reward is  : 9.3
Ep: 4000 last 100 episodes reward is  : 9.37
Ep: 4400 last 100 episodes reward is  : 9.36
Ep: 4800 last 100 episodes reward is  : 9.41
Ep: 5200 last 100 episodes reward is  : 9.31
Ep: 5600 last 100 episodes reward is  : 9.38
Ep: 6000 last 100 episodes reward is  : 9.4
Ep: 6400 last 100 episodes reward is  : 9.3
Ep: 6800 last 100 episodes reward is  : 9.47
Ep: 7200 last 100 episodes reward is  : 9.31
Ep: 7600 last 100 episodes reward is  : 9.36
Ep: 8000 last 100 episodes reward is  : 9.5
Ep: 8400 last 100 episodes re

score,▁
score,14995928.10722


wandb: Agent Starting Run: hfsintoc with config:
wandb: 	LR: 0.004698292646577228
wandb: 	network_size: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'LR': 0.004698292646577228, 'network_size': 128} Its just getting started 
Ep: 400 last 100 episodes reward is  : 403.01
problem solved at episode 424
{'LR': 0.004698292646577228, 'network_size': 128} 92589.81004913758
Ep: 400 last 100 episodes reward is  : 152.97
Ep: 800 last 100 episodes reward is  : 446.21
Ep: 1200 last 100 episodes reward is  : 94.1
problem solved at episode 1456
{'LR': 0.004698292646577228, 'network_size': 128} 397661.93824103306
Ep: 400 last 100 episodes reward is  : 81.48
Ep: 800 last 100 episodes reward is  : 88.35
problem solved at episode 1103
{'LR': 0.004698292646577228, 'network_size': 128} 394229.5553144145
problem solved at episode 394
{'LR': 0.004698292646577228, 'network_size': 128} 143361.9515457032
Ep: 400 last 100 episodes reward is  : 140.3
Ep: 800 last 100 episodes reward is  : 142.57
Ep: 1200 last 100 episodes reward is  : 226.06
Ep: 1600 last 100 episodes reward is  : 103.25
Ep: 2000 last 100 episodes reward is  : 110.19
Ep: 2400 last 100 episod

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
'''
plt.plot(total_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')

plt.show()'''
